# Intrusion detection / prevention system

### DoS (denial of service) attacks are cyberattacks that aim to make a machines, servces, or networks unavailable to their intended users by overwhelming it with a flood of traffic

how DoS attacks work
- overwhelming resources
- disrupting operations
- exploiting vulnerabilities

common methods
- volume based attacks : uses floods of traffic to consume all the bandwidth
- protocol attacks : attack exploits vulnerabilities in the network protocol to consume server resources

Symptoms of attacks
- unusually slow network performance
- inability to accerss a specific website or online service

## goal
using the CIC-IDS2017 dataset available at https://www.unb.ca/cic/datasets/ids-2017.html.
we will build a binary classicication model using supervised learning to be able to predict if a network is being targeted for a DoS attack or not

## dataset
- samples : 55,000
- features : 78
- target : Binary labels (Benign vs DoS)

## Team

Jose Diaz id:708095
add()

## Extract

script to extract data from csv, json and parquet files

In [1]:
#LIBRARIES
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix

In [2]:
from Extract import extract

folder = 'data/'
data = extract(folder=folder)

loaded ids_0.csv of shape : (1001, 79)
loaded ids_1.csv of shape : (1001, 79)
loaded ids_2.csv of shape : (1001, 79)
loaded ids_3.json of shape : (1001, 79)
loaded ids_4.json of shape : (1001, 79)
loaded ids_5.parquet of shape : (15001, 79)
loaded ids_6.parquet of shape : (5001, 79)
loaded ids_7.json of shape : (9000, 79)
loaded ids_8.parquet of shape : (10293, 79)
loaded ids_9.json of shape : (10293, 79)


## Transform - Barebones implementation

In [3]:
#Converting our saved data into a pandas dataframe
data = pd.DataFrame(data)

In [4]:
#Viewing columns of our dataframe
#We notice that there are spaces in the beginning of our DataFrame, we will remove these later
print(f"Original DataFrame columns: {data.columns.tolist()}")

Original DataFrame columns: [' Destination Port', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count', ' 

In [5]:
#Viewing total amount of columns in our dataframe
print(f"Total Number of columns in DataFrame: {len(data.columns.tolist())}")

Total Number of columns in DataFrame: 79


In [6]:
#Stripping every beginning space from every column in our dataframe
data.columns = data.columns.str.lstrip()
print(f"Columns after stripping beggining space from all columns in DataFrame: \n\n{data.columns.tolist()}")

Columns after stripping beggining space from all columns in DataFrame: 

['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Coun

In [7]:
#displaying the shape of our data as well as the first 5 rows
print("Shape of 'data' DataFrame: ",data.shape, "\n")
print("First 5 rows of 'data DataFrame: ")
data.head()

Shape of 'data' DataFrame:  (54593, 79) 

First 5 rows of 'data DataFrame: 


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,36102,23,2,1,31,6,31,0,15.5,21.920310,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,53,30805,2,2,108,230,54,54,54.0,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,443,5535509,8,8,372,3876,191,0,46.5,71.876879,...,20,346981.0,0.0,346981,346981,5188524.0,0.0,5188524,5188524,BENIGN
3,53,49235,4,2,152,202,38,38,38.0,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,53,181,2,2,70,218,35,35,35.0,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [8]:
#Displaying the last 5 rows of DataFrame
data.tail()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
54588,80,11512204,8,5,326,11632,326,0,40.750000,115.258405,...,32,892.0,0.0,892,892,6507197.0,0.0,6507197,6507197,DoS GoldenEye
54589,80,11513325,5,5,471,3525,471,0,94.200000,210.637604,...,32,918.0,0.0,918,918,6508582.0,0.0,6508582,6508582,DoS GoldenEye
54590,80,11509201,7,6,314,11632,314,0,44.857143,118.680845,...,32,899.0,0.0,899,899,6503248.0,0.0,6503248,6503248,DoS GoldenEye
54591,80,11509095,8,5,369,11632,369,0,46.125000,130.461201,...,32,914.0,0.0,914,914,6504954.0,0.0,6504954,6504954,DoS GoldenEye
54592,80,11512230,5,5,672,3525,672,0,134.400000,300.527536,...,32,821.0,0.0,821,821,6506213.0,0.0,6506213,6506213,DoS GoldenEye


In [9]:
# display datas type and info of dataframe
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54593 entries, 0 to 54592
Data columns (total 79 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Destination Port             54593 non-null  int64  
 1   Flow Duration                54593 non-null  int64  
 2   Total Fwd Packets            54593 non-null  int64  
 3   Total Backward Packets       54593 non-null  int64  
 4   Total Length of Fwd Packets  54593 non-null  int64  
 5   Total Length of Bwd Packets  54593 non-null  int64  
 6   Fwd Packet Length Max        54593 non-null  int64  
 7   Fwd Packet Length Min        54593 non-null  int64  
 8   Fwd Packet Length Mean       54593 non-null  float64
 9   Fwd Packet Length Std        54593 non-null  float64
 10  Bwd Packet Length Max        54593 non-null  int64  
 11  Bwd Packet Length Min        54593 non-null  int64  
 12  Bwd Packet Length Mean       54593 non-null  float64
 13  Bwd Packet Lengt

In [10]:
# display total number of empty values in dataframe
data.isna().sum().sum()

np.int64(150)

In [11]:
# removing samples with missing values in dataframe
data = data.dropna()
data.isna().sum().sum()

np.int64(0)

In [12]:
#list columns in dataframe that contain only "String" values --> 'object' is used to define String
columns_with_str = []
for col in data.columns:
    if data[col].dtype == "object":
        columns_with_str.append(col)
print("Columns that contain string values in DataFrame: ")
print(columns_with_str)

Columns that contain string values in DataFrame: 
['Label']


In [13]:
#Display String values within 'label' column in DataFrame
data['Label'].unique()

array(['BENIGN', 'DoS Hulk', 'DoS GoldenEye'], dtype=object)

In [14]:
"""
Up to this point, now that we have organized our DataFrame, and somewhat familiarized with our data it is clear that this is a supervised learning model because:
    - We understand that our data contains 79 columns, one of which are named 'Label'
    - After inspecting 'Label' column, we see that it contains 3 unique String values being used to categorize our samples
    - 'Label': BENIGN, DoS Hulk, DoS GoldenEye
    - This means we have 78 feature columns, and 1 Label(known output) column

    
    - BENIGN: Refers to normal, legitimate network traffic or activity that does not pose a threat to a system. It serves as a baseline for normal behavior
              when training intrusion detection systems.
    - Dos Hulk: A high-volume DoS attack where the attacker generates a huge volume of HTTP requests to consume all of the servers resources,
                such as bandwidth and processing power.
    - DoS GoldenEye: This is a type of application-layer DoS attack that aims to keep connections alive for a long time by sending slow, randomized HTTP
                    requests using cache control and keep-alive headers. This gradually consumes all available server connections and resources, making
                    the server unresponsive to legitimate users. Its slow and persistent nature makes it more difficult to detect than a volumetric attack
                    like DoS Hulk. The attack tool is an open-source Python script available on GitHub
"""

"\nUp to this point, now that we have organized our DataFrame, and somewhat familiarized with our data it is clear that this is a supervised learning model because:\n    - We understand that our data contains 79 columns, one of which are named 'Label'\n    - After inspecting 'Label' column, we see that it contains 3 unique String values being used to categorize our samples\n    - 'Label': BENIGN, DoS Hulk, DoS GoldenEye\n    - This means we have 78 feature columns, and 1 Label(known output) column\n\n\n    - BENIGN: Refers to normal, legitimate network traffic or activity that does not pose a threat to a system. It serves as a baseline for normal behavior\n              when training intrusion detection systems.\n    - Dos Hulk: A high-volume DoS attack where the attacker generates a huge volume of HTTP requests to consume all of the servers resources,\n                such as bandwidth and processing power.\n    - DoS GoldenEye: This is a type of application-layer DoS attack that aims

In [15]:
# we also want to make clear what our labels are so we will assign attack as the new name
data = data.rename(columns={"Label": "attack"})
data.columns[-1]

'attack'

In [16]:
# viewing the different type of attacks
data['attack'].unique()

array(['BENIGN', 'DoS Hulk', 'DoS GoldenEye'], dtype=object)

In [17]:
# we group the dos attacks by assigning 1 if we see dos in the name of the attack label
data['attack'] = data['attack'].apply(lambda x: 1 if "dos" in str(x).lower() else 0 )
data['attack'].tail()

54588    1
54589    1
54590    1
54591    1
54592    1
Name: attack, dtype: int64

In [18]:
# well store our true values for our supervised learning model in y to pass into our test train split function later on
y = data['attack'].copy()

In [19]:
#calculating the variance for each column
column_variances = data.var()

# remove data which contains low to no variance
filtered_data = data.loc[:, column_variances > 1].copy()
for x, col in enumerate(filtered_data.columns):
    print(f"{x:<{2}} {col:<{30}} var : {filtered_data[col].var():>{25}.5f}")

0  Destination Port               var :            41914517.35844
1  Flow Duration                  var :    1830912789519646.50000
2  Total Fwd Packets              var :                 795.60425
3  Total Backward Packets         var :                1983.40707
4  Total Length of Fwd Packets    var :             1064269.43731
5  Total Length of Bwd Packets    var :          5878887504.61399
6  Fwd Packet Length Max          var :               92521.46433
7  Fwd Packet Length Min          var :                 226.87373
8  Fwd Packet Length Mean         var :                6115.42847
9  Fwd Packet Length Std          var :               15095.76915
10 Bwd Packet Length Max          var :            10902763.51105
11 Bwd Packet Length Min          var :                 738.08578
12 Bwd Packet Length Mean         var :              926778.79230
13 Bwd Packet Length Std          var :             2153990.96941
14 Flow IAT Mean                  var :     295608651273710.18750
15 Flow IA

In [20]:
# displaying some information about the data which has been removed to check if there is any information we should reconsider
removed_data = data.loc[:, column_variances <= 1 ]
removed_data

,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,attack
0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54588,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
54589,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1
54590,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
54591,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


In [21]:
# viewing if our data is standardized (mean = 0, std =1 )
filtered_data.describe()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,54475.000000,5.447500e+04,54475.000000,54475.000000,54475.000000,5.447500e+04,54475.000000,54475.000000,54475.000000,54475.000000,...,54475.000000,54475.000000,5.447500e+04,5.447500e+04,5.447500e+04,5.447500e+04,5.447500e+04,5.447500e+04,5.447500e+04,5.447500e+04
mean,935.357779,4.032098e+07,5.804920,4.483947,368.405672,7.608740e+03,265.691363,2.115888,52.745352,107.498522,...,1.573125,29.047306,8.619742e+04,1.377269e+04,1.008245e+05,7.705709e+04,3.743201e+07,4.631730e+05,3.776827e+07,3.709637e+07
std,6474.142210,4.278917e+07,28.206458,44.535459,1031.634353,7.667390e+04,304.173412,15.062328,78.201205,122.864841,...,26.263925,5.217411,5.805961e+05,2.332652e+05,6.785944e+05,5.492533e+05,4.305943e+07,5.066017e+06,4.334249e+07,4.307849e+07
min,0.000000,0.000000e+00,1.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,80.000000,1.472035e+05,2.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,32.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,80.000000,1.180912e+07,6.000000,5.000000,340.000000,1.159500e+04,329.000000,0.000000,47.571429,126.019707,...,1.000000,32.000000,6.000000e+00,0.000000e+00,6.000000e+00,6.000000e+00,6.801986e+06,0.000000e+00,6.801986e+06,6.759334e+06
75%,80.000000,8.582139e+07,8.000000,6.000000,396.000000,1.159500e+04,374.000000,0.000000,66.500000,156.624152,...,2.000000,32.000000,1.040000e+03,0.000000e+00,1.040000e+03,1.033000e+03,8.530000e+07,0.000000e+00,8.540000e+07,8.530000e+07
max,63849.000000,1.199990e+08,5117.000000,8350.000000,88632.000000,1.450000e+07,11595.000000,1983.000000,2319.000000,5185.441640,...,5028.000000,48.000000,1.370000e+07,6.905038e+06,1.920000e+07,1.370000e+07,1.200000e+08,6.070000e+07,1.200000e+08,1.200000e+08


In [22]:
filtered_data.columns

Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow IAT Mean', 'Flow IAT Std',
       'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean',
       'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total',
       'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min',
       'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s',
       'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length',
       'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance',
       'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size',
       'Fwd Header Length.1', 'Subflow Fwd Packets', 'Subflow Fwd Bytes',
 

In [23]:
# destination port converted to categories
filtered_data['Destination Port'] = filtered_data['Destination Port'].astype('category')

In [24]:
# x will contain the information outside of the labels for each sample
x = filtered_data.copy()

In [25]:
# y will hold only the label values which are stored as true false or 0,1 where the label is considered an attack
y

0        0
1        0
2        0
3        0
4        0
        ..
54588    1
54589    1
54590    1
54591    1
54592    1
Name: attack, Length: 54475, dtype: int64

In [26]:
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=.3,random_state=42,stratify=y)

In [27]:
#creating a standard scaler to standardize the filtered data
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [28]:
# displaying if x train is standardized (mean = 0, std = 1)
pd.DataFrame(X_train).describe()

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
count,3.813200e+04,3.813200e+04,3.813200e+04,3.813200e+04,3.813200e+04,3.813200e+04,3.813200e+04,3.813200e+04,3.813200e+04,3.813200e+04,...,3.813200e+04,3.813200e+04,3.813200e+04,3.813200e+04,3.813200e+04,3.813200e+04,3.813200e+04,3.813200e+04,3.813200e+04,3.813200e+04
mean,-1.416166e-17,-8.273392e-17,-4.844779e-18,-8.198857e-18,1.490701e-18,7.453506e-19,-7.565309e-17,1.267096e-17,2.254686e-17,-8.944207e-18,...,-4.099428e-18,-2.724256e-16,-1.621138e-17,-1.267096e-17,2.608727e-18,1.378899e-17,-8.217490e-17,1.863377e-17,-1.155293e-17,-5.981439e-17
std,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,...,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00
min,-1.441162e-01,-9.448165e-01,-1.512844e-01,-8.716127e-02,-3.331995e-01,-8.658796e-02,-8.604339e-01,-1.385699e-01,-6.618441e-01,-8.603899e-01,...,-5.225008e-02,-5.580236e+00,-1.489493e-01,-5.959565e-02,-1.486489e-01,-1.404612e-01,-8.708927e-01,-9.221932e-02,-8.730392e-01,-8.625685e-01
25%,-1.317000e-01,-9.413399e-01,-1.200244e-01,-8.716127e-02,-3.331995e-01,-8.658796e-02,-8.604339e-01,-1.385699e-01,-6.618441e-01,-8.603899e-01,...,-5.225008e-02,5.642415e-01,-1.489493e-01,-5.959565e-02,-1.486489e-01,-1.404612e-01,-8.708927e-01,-9.221932e-02,-8.730392e-01,-8.625685e-01
50%,-1.317000e-01,-6.683573e-01,5.015433e-03,8.218268e-03,-2.803391e-02,4.311690e-02,2.028614e-01,-1.385699e-01,-6.622808e-02,1.469908e-01,...,-2.003978e-02,5.642415e-01,-1.489390e-01,-5.959565e-02,-1.486402e-01,-1.404504e-01,-7.126335e-01,-9.221932e-02,-7.158321e-01,-7.050774e-01
75%,-1.317000e-01,1.062092e+00,6.753536e-02,2.729418e-02,2.222867e-02,4.311690e-02,3.482970e-01,-1.385699e-01,1.708123e-01,3.926971e-01,...,1.217051e-02,5.642415e-01,-1.471727e-01,-5.959565e-02,-1.471395e-01,-1.385917e-01,1.109256e+00,-9.221932e-02,1.098562e+00,1.116685e+00
max,9.765405e+00,1.861340e+00,1.597747e+02,1.591967e+02,7.921809e+01,1.621144e+02,3.661346e+01,1.302651e+02,2.834043e+01,4.056512e+01,...,1.619011e+02,3.636480e+00,2.323187e+01,2.860383e+01,2.768902e+01,2.462948e+01,1.918050e+00,1.179286e+01,1.897360e+00,1.925113e+00


In [29]:
# displayed if x test is standardized (mean = 0, std = 1)
pd.DataFrame(X_test).describe()

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
count,16343.000000,16343.000000,16343.000000,16343.000000,16343.000000,16343.000000,16343.000000,16343.000000,16343.000000,16343.000000,...,16343.000000,16343.000000,16343.000000,16343.000000,16343.000000,16343.000000,16343.000000,16343.000000,16343.000000,16343.000000
mean,0.003512,-0.006390,-0.003609,-0.005420,-0.008461,-0.004915,-0.005819,0.001909,-0.007303,-0.005348,...,-0.005264,-0.009066,-0.006141,-0.008080,-0.008218,-0.003800,-0.003094,-0.005100,-0.003653,-0.002649
std,1.015917,1.002042,0.508093,0.269153,0.724216,0.344640,0.942331,0.968007,0.924659,0.937056,...,0.228429,1.006028,0.969191,0.889943,0.945171,0.976682,1.002502,0.972821,1.002105,1.002470
min,-0.144116,-0.944816,-0.151284,-0.087161,-0.333200,-0.086588,-0.860434,-0.138570,-0.661844,-0.860390,...,-0.052250,-5.580236,-0.148949,-0.059596,-0.148649,-0.140461,-0.870893,-0.092219,-0.873039,-0.862569
25%,-0.131700,-0.941517,-0.120024,-0.087161,-0.333200,-0.086588,-0.860434,-0.138570,-0.661844,-0.860390,...,-0.052250,-1.739938,-0.148949,-0.059596,-0.148649,-0.140461,-0.870893,-0.092219,-0.873039,-0.862569
50%,-0.131700,-0.669790,0.005015,0.008218,-0.028034,0.043117,0.202861,-0.138570,-0.066898,0.146162,...,-0.020040,0.564241,-0.148939,-0.059596,-0.148640,-0.140450,-0.713670,-0.092219,-0.716862,-0.706640
75%,-0.131700,1.062271,0.067535,0.027294,0.021331,0.043117,0.351529,-0.138570,0.167150,0.390286,...,-0.020040,0.564241,-0.147184,-0.059596,-0.147149,-0.138599,1.111581,-0.092219,1.098562,1.119008
max,9.709687,1.858674,59.117611,28.774687,41.255489,39.946127,27.218321,95.083106,23.506721,27.148556,...,17.437941,2.868421,18.623970,27.573338,22.034493,19.747812,1.918050,11.792856,1.897360,1.925113


In [30]:
(pd.DataFrame(filtered_data).corr().style.background_gradient(cmap='PRGn',vmin=-1,vmax=1))

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
Destination Port,1.000000,-0.106448,-0.016138,-0.009253,0.079789,-0.012437,0.082552,0.053986,0.150601,0.088189,-0.149435,-0.011710,-0.159799,-0.145269,-0.056410,-0.082127,-0.100051,-0.033474,-0.100519,-0.068084,-0.081616,-0.099373,-0.036886,-0.036536,-0.036309,-0.030177,-0.030445,-0.006219,-0.025438,-0.015682,0.038189,0.199585,0.017930,-0.133253,-0.137524,-0.137254,-0.106951,-0.134923,0.150601,-0.159799,-0.025438,-0.016138,0.079789,-0.009253,-0.012448,-0.068831,0.169803,-0.003060,-0.079979,-0.017713,-0.005825,-0.017121,-0.017189,-0.098345,-0.008447,-0.098388,-0.097587
Flow Duration,-0.106448,1.000000,0.054965,0.046384,0.096871,0.049202,0.226090,-0.122551,0.146729,0.226769,0.345293,-0.150817,0.389194,0.275648,0.412171,0.852133,0.982626,0.177666,0.998802,0.577491,0.863248,0.982543,0.248615,0.402092,0.320450,0.396930,0.408405,0.034532,0.067807,0.067086,-0.278838,-0.054940,-0.161338,0.350877,0.433108,0.350416,0.230524,0.427050,0.146729,0.389194,0.067807,0.054965,0.096871,0.046384,0.049236,-0.462161,-0.022639,0.034167,-0.388463,-0.043121,0.033067,-0.018596,-0.054150,0.975594,0.154850,0.982222,0.962073
Total Fwd Packets,-0.016138,0.054965,1.000000,0.980671,0.555802,0.987946,0.072699,-0.015322,0.028210,0.046537,0.081797,-0.014420,0.081862,0.061191,-0.032076,0.020137,0.024064,-0.034404,0.051871,-0.028226,0.030026,0.023693,-0.037835,0.055736,0.011601,0.010350,0.014795,-0.003001,0.984720,0.979701,-0.039125,-0.008734,-0.022590,0.082155,0.081553,0.067284,0.046393,0.077760,0.028210,0.081862,0.984720,1.000000,0.555802,0.980671,0.987972,0.042897,0.004630,0.971577,-0.026753,0.050443,0.050295,0.066826,0.038713,0.019824,0.010599,0.020662,0.018801
Total Backward Packets,-0.009253,0.046384,0.980671,1.000000,0.546601,0.996089,0.055311,-0.007785,0.021122,0.035870,0.054844,0.000167,0.054584,0.033826,-0.022321,0.019625,0.020942,-0.025677,0.044710,-0.015456,0.029183,0.020650,-0.026570,0.061671,0.017803,0.021659,0.026404,-0.002341,0.939263,0.997975,-0.029721,-0.003937,-0.011876,0.055117,0.066628,0.044463,0.026405,0.064027,0.021122,0.054584,0.939263,0.980671,0.546601,1.000000,0.996093,0.022525,0.004691,0.990361,-0.034776,0.018328,0.023549,0.028793,0.013710,0.018719,0.004348,0.018948,0.018279
Total Length of Fwd Packets,0.079789,0.096871,0.555802,0.546601,1.000000,0.525851,0.546494,0.058952,0.594986,0.507964,0.139248,-0.037884,0.138430,0.121164,-0.071687,0.064213,0.045135,-0.091060,0.088467,-0.049264,0.064948,0.044283,-0.072722,0.155937,0.102290,0.090829,0.091517,0.003960,0.549550,0.560682,-0.103861,-0.018963,-0.033276,0.170176,0.190179,0.154499,0.094396,0.184974,0.594986,0.138430,0.549550,0.555802,1.000000,0.546601,0.525968,0.153650,0.021577,0.553471,-0.026110,0.215722,0.208428,0.264171,0.169686,0.039368,0.026208,0.041969,0.036660
Total Length of Bwd Packets,-0.012437,0.049202,0.987946,0.996089,0.525851,1.000000,0.048755,-0.013593,0.012761,0.030474,0.074546,-0.006823,0.075325,0.053853,-0.021347,0.024931,0.028535,-0.025309,0.047409,-0.014490,0.036442,0.028259,-0.027830,0.052177,0.014129,0.019587,0.0236

In [31]:
np.unique(y_test,return_counts=True)

(array([0, 1]), array([ 1502, 14841]))

In [32]:
model = LogisticRegression(random_state=42)
model.fit(X_train,y_train)

predictions = model.predict(X_test)

accuracy = accuracy_score(y_test,predictions)

print("model accuracy : ", accuracy)

model accuracy :  0.9945542434069632


In [33]:
xg_model = XGBClassifier(n_estimators=1).fit(X_train, y_train, verbose = True)
xg_pred = xg_model.predict(X_test)

xg_model

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [34]:
xg_pred

array([1, 1, 1, ..., 1, 1, 1], shape=(16343,))

In [35]:
# compute the measures
acc_xgboost1 = accuracy_score(y_test, xg_pred)
f1score_xgboost1 = f1_score(y_test, xg_pred)
precision_xgboost1 = precision_score(y_test, xg_pred)
recall_xgboost1 = recall_score(y_test, xg_pred)

# display the measures

print('-'*25)
print('XGBoost Model: 66 features')
print('-'*25)
print('Accuracy: {:.3f}'.format(acc_xgboost1))
print('F1-score: {:.3f}'.format(f1score_xgboost1))
print('Precision: {:.3f}'.format(precision_xgboost1))
print('Recall: {:.3f}'.format(recall_xgboost1))

-------------------------
XGBoost Model: 66 features
-------------------------
Accuracy: 0.996
F1-score: 0.998
Precision: 0.996
Recall: 1.000


In [36]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, xg_pred)

print("Confusion Matrix:")
print("-"*18)
print(conf_matrix)
print("-"*18)
print("tn,fp")
print("fn,tp")

Confusion Matrix:
------------------
[[ 1439    63]
 [    3 14838]]
------------------
tn,fp
fn,tp


In [37]:
xg_model.feature_importances_

array([3.1454766e-01, 0.0000000e+00, 0.0000000e+00, 1.3159961e-02,
       0.0000000e+00, 0.0000000e+00, 3.0926039e-04, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       3.8845714e-03, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       6.0501593e-01, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 5.5297695e-02, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 7.7849207e-03, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00], dtype=float32)

In [38]:
print('feature xgboost importance')
for x,row in enumerate(zip(x, xg_model.feature_importances_)):
    print(x,row)

feature xgboost importance
0 ('Destination Port', np.float32(0.31454766))
1 ('Flow Duration', np.float32(0.0))
2 ('Total Fwd Packets', np.float32(0.0))
3 ('Total Backward Packets', np.float32(0.013159961))
4 ('Total Length of Fwd Packets', np.float32(0.0))
5 ('Total Length of Bwd Packets', np.float32(0.0))
6 ('Fwd Packet Length Max', np.float32(0.0003092604))
7 ('Fwd Packet Length Min', np.float32(0.0))
8 ('Fwd Packet Length Mean', np.float32(0.0))
9 ('Fwd Packet Length Std', np.float32(0.0))
10 ('Bwd Packet Length Max', np.float32(0.0))
11 ('Bwd Packet Length Min', np.float32(0.0))
12 ('Bwd Packet Length Mean', np.float32(0.0))
13 ('Bwd Packet Length Std', np.float32(0.0))
14 ('Flow IAT Mean', np.float32(0.0))
15 ('Flow IAT Std', np.float32(0.0))
16 ('Flow IAT Max', np.float32(0.0))
17 ('Flow IAT Min', np.float32(0.0))
18 ('Fwd IAT Total', np.float32(0.0))
19 ('Fwd IAT Mean', np.float32(0.0))
20 ('Fwd IAT Std', np.float32(0.0))
21 ('Fwd IAT Max', np.float32(0.0))
22 ('Fwd IAT Min', np

In [39]:
# were seeing a large dependence on destination port confusing the disporportionate representation of dos attacks to a port as a feature to determine attacks
filtered_data = filtered_data.drop('Destination Port', axis=1)

In [40]:
x2 = filtered_data
X_train, X_test, y_train, y_test = train_test_split(x2, y, test_size=.3, random_state=42, stratify=y)

In [41]:
xg = XGBClassifier(n_estimators=1).fit(X_train, y_train, verbose = True)
xg_pred_2 = xg.predict(X_test)

# compute the measures
acc_xgboost2 = accuracy_score(y_test, xg_pred_2)
f1score_xgboost2 = f1_score(y_test, xg_pred_2)
precision_xgboost2 = precision_score(y_test, xg_pred_2)
recall_xgboost2= recall_score(y_test, xg_pred_2)

# display the measures
print('-'*25)
print('XGBoost Model: removed destination port')
print('-'*25)
print('Accuracy: {:.3f}'.format(acc_xgboost2))
print('F1-score: {:.3f}'.format(f1score_xgboost2))
print('Precision: {:.3f}'.format(precision_xgboost2))
print('Recall: {:.3f}'.format(recall_xgboost2))

-------------------------
XGBoost Model: removed destination port
-------------------------
Accuracy: 0.990
F1-score: 0.995
Precision: 0.990
Recall: 1.000


In [42]:
for i,row in enumerate(zip(xg.feature_importances_, x2)):
    if row[0] == 0:
        continue
    print(i,row[1])

2 Total Backward Packets
3 Total Length of Fwd Packets
5 Fwd Packet Length Max
11 Bwd Packet Length Mean
12 Bwd Packet Length Std
13 Flow IAT Mean
27 Fwd Header Length
31 Min Packet Length
45 Init_Win_bytes_backward
46 act_data_pkt_fwd


In [ ]:
#create a total fwd bwd ration column
filtered_data['f_b_total_packet_ratio'] =  filtered_data['Total Backward Packets'] / (1 + filtered_data['Total Length of Fwd Packets'])

#create a wanted list
wanted = ['f_b_total_packet_ratio', 'Fwd Packet Length Max', 'Bwd Packet Length Std', 'Flow IAT Mean', 'act_data_pkt_fwd']

#create a new dataframe only using the wannted list
df = filtered_data[wanted].copy()

In [55]:
X_train, X_test, y_train, y_test = train_test_split(df,y,test_size=.3,random_state=42,stratify=y)

In [ ]:
final_xg = XGBClassifier(n_estimators=1).fit(X_train,y_train)
pred3 = final_xg.predict(X_test)

# compute the measures
acc_xgboost3 = accuracy_score(y_test, pred3)
f1score_xgboost3 = f1_score(y_test, pred3)
precision_xgboost3 = precision_score(y_test, pred3)
recall_xgboost3 = recall_score(y_test, pred3)

# display the measures
print('-'*25)
print('XGBoost Model: reduced')
print('-'*25)
print('Accuracy: {:.3f}'.format(acc_xgboost3))
print('F1-score: {:.3f}'.format(f1score_xgboost3))
print('Precision: {:.3f}'.format(precision_xgboost3))
print('Recall: {:.3f}'.format(recall_xgboost3))

-------------------------
XGBoost Model: removed destination port
-------------------------
Accuracy: 0.983
F1-score: 0.991
Precision: 0.985
Recall: 0.997
